## Inisialisasi Library untuk Tahap Inference

- **Import library**
  - `pickle` → membaca model dan artefak yang sudah disimpan.  
  - `numpy` → manipulasi array dan operasi numerik.  
- **Nonaktifkan peringatan** untuk menjaga output tetap bersih saat proses inference.

In [1]:
# Mengimpor library untuk membaca file model dan artefak yang disimpan
import pickle

# Mengimpor library untuk manipulasi array dan operasi numerik
import numpy as np

# Menonaktifkan peringatan agar output lebih bersih
import warnings
warnings.filterwarnings("ignore")

print("Library yang dibutuhkan untuk inference berhasil di-import.")

Library yang dibutuhkan untuk inference berhasil di-import.


## Memuat Model

Sebelum melakukan **inference**, kita perlu memuat kembali artefak yang sebelumnya disimpan di `../Models`:

- **Model Ensemble Terbaik** (`best_ensemble_model.pkl`) → model pemenang hasil evaluasi.
- **TF-IDF Vectorizer** (`tfidf_vectorizer.pkl`) → mengubah teks mentah menjadi vektor numerik.
- **Feature Selector** (`feature_selector.pkl`) → memilih fitur paling relevan dari hasil TF-IDF.

Semua komponen ini akan digunakan bersama untuk memproses teks baru dan menghasilkan prediksi.


In [2]:
# Membuka dan memuat model ensemble terbaik dari file pickle
with open("../Models/best_ensemble_model.pkl", "rb") as f:
    best_model = pickle.load(f)

# Memuat kembali TF-IDF Vectorizer untuk mentransformasi teks
with open("../Models/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

# Memuat kembali Feature Selector untuk memilih fitur yang relevan
with open("../Models/feature_selector.pkl", "rb") as f:
    feature_selector = pickle.load(f)

print("Model terbaik, TF-IDF Vectorizer, dan Feature Selector berhasil dimuat.")

Model terbaik, TF-IDF Vectorizer, dan Feature Selector berhasil dimuat.


## Preprocessing Untuk Data Baru

Tahap ini mempersiapkan teks mentah sebelum diprediksi oleh model terbaik.  
Proses yang dilakukan dalam fungsi `preprocess_for_inference` meliputi:

1. **Case Folding** → mengubah semua huruf menjadi lowercase.
2. **Normalisasi Slang** → mengganti kata tidak baku berdasarkan kamus `slangwords.json`.
3. **Stopword Removal** → menghapus kata umum yang tidak membawa makna signifikan.
4. **Tokenisasi & Filtering** → memisahkan teks menjadi kata dan membuang karakter non-alfanumerik.
5. **Stemming** → mengubah kata ke bentuk dasarnya menggunakan *Sastrawi Stemmer*.

Hasil akhir preprocessing ini kemudian:
- **Ditransformasikan** ke bentuk vektor numerik menggunakan **TF-IDF Vectorizer**.
- **Disaring** menggunakan **Feature Selector** agar hanya fitur relevan yang dipakai untuk prediksi.

In [ ]:
import re
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Data teks baru yang akan digunakan untuk prediksi sentimen
new_texts = [
    "Produk ini sangat bagus!",                      # Ekspektasi: positif
    "Pelayanan sangat lambat dan mengecewakan.",      # Ekspektasi: negatif
    "Harganya cukup mahal tapi kualitasnya biasa saja." # Ekspektasi: negatif (karena model biner)
]

# --- Fungsi Tunggal untuk Preprocessing Teks Inference ---
def preprocess_for_inference(texts):
    # 1. Muat alat preprocessing yang diperlukan
    with open('../Datasets/slangwords.json', 'r') as file:
        slang_dict = json.load(file)
    stop_words = set(stopwords.words('indonesian'))
    stemmer = StemmerFactory().create_stemmer()

    # 2. Proses setiap teks dalam list
    processed_texts = []
    for text in texts:
        text = text.lower() # Case folding
        # Normalisasi slang
        words = text.split()
        normalized_words = [slang_dict.get(word, word) for word in words]
        text = ' '.join(normalized_words)
        # Hapus stopwords
        words = word_tokenize(text)
        filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
        text = ' '.join(filtered_words)
        # Stemming
        text = stemmer.stem(text)
        processed_texts.append(text)
    
    return processed_texts

# --- Jalankan Pipeline ---
# 1. Bersihkan teks mentah
preprocessed_texts = preprocess_for_inference(new_texts)
print("Teks setelah preprocessing:")
for i, text in enumerate(preprocessed_texts):
    print(f"{i+1}. {text}")

# 2. Terapkan Vectorizer dan Feature Selector yang sudah dimuat
new_tfidf = tfidf_vectorizer.transform(preprocessed_texts)
new_features_selected = feature_selector.transform(new_tfidf)

print(f"\nDimensi fitur setelah TF-IDF dan Feature Selection: {new_features_selected.shape}")
print("Data baru siap untuk diprediksi.")

Teks setelah preprocessing:
1. produk bagus
2. layan lambat kecewa
3. harga mahal kualitas

Dimensi fitur setelah TF-IDF dan Feature Selection: (3, 2000)
Data baru siap untuk diprediksi.


## Prediksi Sentimen

Pada tahap ini, **model terbaik** yang telah dimuat digunakan untuk memprediksi sentimen data teks baru yang sudah melalui preprocessing, TF-IDF, dan feature selection.

Output yang dihasilkan:
- **Format numerik**: `1` untuk sentimen positif, `0` untuk sentimen negatif.

In [ ]:
# Menggunakan model terbaik yang telah dimuat untuk memprediksi sentimen
predictions = best_model.predict(new_features_selected)

print("Prediksi berhasil dilakukan.")
print("Hasil prediksi (dalam format numerik):", predictions)

Prediksi berhasil dilakukan.
Hasil prediksi (dalam format numerik): [1 0 0]


## Konversi Prediksi ke Label Sentimen

Tahap ini mengubah hasil prediksi **numerik** (`0` = negatif, `1` = positif) menjadi **label string** yang lebih mudah dibaca.  
Hasil akhir ditampilkan per kalimat input, lengkap dengan teks aslinya.

In [ ]:
# Mapping label numerik (0 dan 1) ke label string yang mudah dibaca.
label_map = {0: "negatif", 1: "positif"}

# Menampilkan hasil prediksi dari model terbaik untuk setiap kalimat input.
print("\n--- Hasil Prediksi Sentimen ---")
for i, text in enumerate(new_texts):
    predicted_label_numeric = predictions[i]
    predicted_label_string = label_map[predicted_label_numeric]
    
    print(f"\nTeks Asli: \"{text}\"")
    print(f"Prediksi Sentimen: {predicted_label_string.upper()}")
    print("-" * 40)


--- Hasil Prediksi Sentimen ---

Teks Asli: "Produk ini sangat bagus!"
Prediksi Sentimen: POSITIF
----------------------------------------

Teks Asli: "Pelayanan sangat lambat dan mengecewakan."
Prediksi Sentimen: NEGATIF
----------------------------------------

Teks Asli: "Harganya cukup mahal tapi kualitasnya biasa saja."
Prediksi Sentimen: NEGATIF
----------------------------------------
